# 02_03 CNN

In [ ]:
import pandas as pd
import numpy as np
from tensorflow.contrib import keras

from smh_eyetracking.features02 import config as config_features02
from smh_eyetracking.features02.utils.features02_dlib import FEATURES, TARGETS
from smh_eyetracking.keras import config as config_keras
from smh_eyetracking.keras import losses
from smh_eyetracking.utils import data_model

## Load data

In [ ]:
data, imgs_left, imgs_right = data_model.load(
    config_features02.PATH_DATA_FEATURES02_DLIB_AUGMENTED_NORM_CSV,
    config_features02.PATH_DATA_FEATURES02_DLIB_AUGMENTED_NORM_IMGS_LEFT,
    config_features02.PATH_DATA_FEATURES02_DLIB_AUGMENTED_NORM_IMGS_RIGHT
)

## Split data

In [ ]:
(
    (train_data, train_imgs_left, train_imgs_right),
    (validation_data, validation_imgs_left, validation_imgs_right),
    (test_data, test_imgs_left, test_imgs_right)
) = data_model.split(
    data, imgs_left, imgs_right,
    validation_size=0.15,
    test_size=0.15
)

In [ ]:
print("Train length: {}".format(len(train_data)))
print("Validation length: {}".format(len(validation_data)))
print("Test length: {}".format(len(test_data)))

## Model

### Architecture

In [ ]:
img_width, img_height = config_features02.FEATURES02_EYES_SIZE
img_shape = (img_height, img_width)

def get_model():

    # Inputs
    left_imgs = keras.layers.Input(shape=img_shape, name='left_imgs', dtype='float32')
    right_imgs = keras.layers.Input(shape=img_shape, name='right_imgs', dtype='float32')
    features = keras.layers.Input(shape=(len(FEATURES),), name='features', dtype='float32')
    
    
    # Model X

    # Features
    xf1 = keras.layers.Dense(256, activation=keras.activations.relu)(features)
    xf2 = keras.layers.Dense(128, activation=keras.activations.relu)(xf1)
    # Left imgs
    xl1 = keras.layers.Conv2D(32, (3, 3), (2,2), activation='relu', padding='same')(
        keras.layers.Reshape((24,32,1))(left_imgs)
    )
    xl2 = keras.layers.Conv2D(64, (3, 3), (2,2), activation='relu', padding='same')(xl1)
    # Right imgs
    xr1 = keras.layers.Conv2D(32, (3, 3), (2,2), activation='relu', padding='same')(
        keras.layers.Reshape((24,32,1))(right_imgs)
    )
    xr2 = keras.layers.Conv2D(64, (3, 3), (2,2), activation='relu', padding='same', input_shape=(24, 32, 1))(xr1)
    # Concatenate
    xconcat = keras.layers.Concatenate()([
        keras.layers.Flatten()(xl2),
        keras.layers.Flatten()(xr2),
        xf2
    ])
    # Dense
    xd_1 = keras.layers.Dense(256, activation=keras.activations.relu)(xconcat)
    xd_2 = keras.layers.Dense(128, activation=keras.activations.relu)(xd_1)
    xd_3 = keras.layers.Dense(32, activation=keras.activations.relu)(xd_2)
    xd_4 = keras.layers.Dense(1)(xd_3)
    
     
    # Model Y
    # Features
    yf1 = keras.layers.Dense(256, activation=keras.activations.relu)(features)
    yf2 = keras.layers.Dense(128, activation=keras.activations.relu)(yf1)
    # Left imgs
    yl1 = keras.layers.Conv2D(32, (3, 3), (2,2), activation='relu', padding='same')(
        keras.layers.Reshape((24,32,1))(left_imgs)
    )
    yl2 = keras.layers.Conv2D(64, (3, 3), (2,2), activation='relu', padding='same')(yl1)
    # Right imgs
    yr1 = keras.layers.Conv2D(32, (3, 3), (2,2), activation='relu', padding='same')(
        keras.layers.Reshape((24,32,1))(right_imgs)
    )
    yr2 = keras.layers.Conv2D(64, (3, 3), (2,2), activation='relu', padding='same', input_shape=(24, 32, 1))(yr1)
    # Concatenate
    yconcat = keras.layers.Concatenate()([
        keras.layers.Flatten()(yl2),
        keras.layers.Flatten()(yr2),
        yf2
    ])
    
    # Dense
    yd_1 = keras.layers.Dense(256, activation=keras.activations.relu)(yconcat)
    yd_2 = keras.layers.Dense(128, activation=keras.activations.relu)(yd_1)
    yd_3 = keras.layers.Dense(32, activation=keras.activations.relu)(yd_2)
    yd_4 = keras.layers.Dense(1)(yd_3)
    
    
    out = keras.layers.Concatenate()([xd_4, yd_4])
    # Model
    model = keras.models.Model(
        inputs=[left_imgs, right_imgs, features],
        outputs=[out]
    )
    return model

### Parameters

In [ ]:
MODEL_NAME = '07-CNNx2-01'

EPOCHS = 50
BATCH_SIZE = 64
LEARNING_RATE = 0.00005
DECAY = 0.000001


LOSS = losses.mean_euclidean

In [ ]:
model = get_model()


model.compile(
    loss=LOSS,
    metrics=[losses.mean_euclidean],
    optimizer=keras.optimizers.Adam(lr=LEARNING_RATE, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=DECAY)
)

print("Parameters to adjust: {}".format(
    np.sum([keras.backend.count_params(p) for p in set(model.trainable_weights)])
))

### Train

In [ ]:
model.fit(
    x={
        'left_imgs':train_imgs_left,
        'right_imgs': train_imgs_right,
        'features': train_data[FEATURES].as_matrix()
    },
    y=train_data[TARGETS].as_matrix(),
    validation_data=(
        {
            'left_imgs': validation_imgs_left,
            'right_imgs': validation_imgs_right,
            'features': validation_data[FEATURES].as_matrix()
        },
        validation_data[TARGETS].as_matrix()
    ),
    epochs=EPOCHS,
    batch_size=BATCH_SIZE,
    verbose=1, callbacks=None, shuffle=True, class_weight=None, sample_weight=None, initial_epoch=0
)

In [ ]:
model.save(config_keras.PATH_MODELS_KERAS+MODEL_NAME)

### Test

In [ ]:
model_test = keras.models.load_model(
    filepath=config_keras.PATH_MODELS_KERAS+MODEL_NAME,
    custom_objects={
        "mean_euclidean": losses.mean_euclidean,
        "ms_euclidean": losses.ms_euclidean,
        "reg_mean_euclidean": losses.reg_mean_euclidean
    }
)

In [ ]:
model.evaluate(
    x={
        'left_imgs':test_imgs_left,
        'right_imgs': test_imgs_right,
        'features': test_data[FEATURES].as_matrix()
    },
    y=test_data[TARGETS].as_matrix(),
    batch_size=1,
    verbose=1, sample_weight=None
)

### Results

Corrected dataset


| Name | Epochs | Batch Size | Learning rate | Decay | Loss | Train | Validation | Test |
|:--:|:--:|:--:|:--:|:--:|:--:|:--:|:--:|:--:|
| 07_CNNx2-01 | 50 | 64| 0.00005 |0.000001 | mean_euclidean | 0.0399 | 0.1807 | 0.1676 |